## 모두를 위한 머신러닝/딥러닝 강의
김성훈 교수님의 모두를 위한 머신러닝/딥러닝 강의 중 lab 강의 코드입니다.
## Lab6 Softmax Classifier
### Example : toy data

In [1]:
import tensorflow as tf
tf.set_random_seed(777) # for reproducibility
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

In [2]:
X = tf.placeholder(dtype = tf.float32, shape = [None, 4])
Y = tf.placeholder(dtype = tf.float32, shape = [None, 3])

nb_classes = 3

W = tf.Variable(initial_value = tf.random_normal(shape = [4, nb_classes]), name = 'weight')
b = tf.Variable(initial_value = tf.random_normal(shape = [nb_classes]), name = 'bias')

In [3]:
# tf.nn.softmax computes softmax activations
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = - tf.reduce_mean(tf.reduce_sum(Y * tf.log(hypothesis), axis = 1))
opt = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize(cost)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y, axis = 1),tf.argmax(hypothesis, axis = 1)), dtype = tf.float32))

In [4]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(10001):
        cost_val, _, acc_val = sess.run(fetches = [cost, opt, accuracy], feed_dict = {X : x_data, Y : y_data})
        if step % 2000 == 0:
            print(step, cost_val, acc_val)
    # Accuracy report        
    h = sess.run(hypothesis, feed_dict = {X : x_data, Y : y_data})
    print(sess.run([tf.argmax(Y, 1),tf.argmax(h,1)], feed_dict={Y : y_data}))
    
    # Testing & One-hot encoding
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.arg_max(a, 1)))

    print('--------------')

    b = sess.run(hypothesis, feed_dict={X: [[1, 3, 4, 3]]})
    print(b, sess.run(tf.arg_max(b, 1)))

    print('--------------')

    c = sess.run(hypothesis, feed_dict={X: [[1, 1, 0, 1]]})
    print(c, sess.run(tf.arg_max(c, 1)))

    print('--------------')

    all = sess.run(hypothesis, feed_dict={
                   X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, sess.run(tf.arg_max(all, 1)))

0 6.92611 0.25
2000 0.457088 0.875
4000 0.356639 0.875
6000 0.304955 1.0
8000 0.268159 1.0
10000 0.239511 1.0
[array([2, 2, 2, 1, 1, 1, 0, 0], dtype=int64), array([2, 2, 2, 1, 1, 1, 0, 0], dtype=int64)]
[[  1.62854176e-02   9.83640373e-01   7.42032644e-05]] [1]
--------------
[[ 0.84297669  0.13680151  0.02022176]] [0]
--------------
[[  3.83510223e-06   3.01761646e-03   9.96978521e-01]] [2]
--------------
[[  1.62854176e-02   9.83640373e-01   7.42032644e-05]
 [  8.42976689e-01   1.36801511e-01   2.02217568e-02]
 [  3.83510223e-06   3.01761623e-03   9.96978521e-01]] [1 0 2]


### Example : zoo data

In [5]:
import sys, os
import numpy as np
os.listdir()

['.ipynb_checkpoints',
 'data-01-test-score.csv',
 'data-02-test-score.csv',
 'data-03-test-score.csv',
 'data-04-test-score.csv',
 'data-05-test-score.csv',
 'diabetes.csv',
 'DLEL.zip',
 'DLEL_1_TensorFlow_Basics.ipynb',
 'DLEL_2_Linear_regression.ipynb',
 'DLEL_3_Minimizing_Cost.ipynb',
 'DLEL_4_2_Loading_Data_from_File.ipynb',
 'DLEL_4_Multi-variable_linear_regression.ipynb',
 'DLEL_5_Logistic_regression.ipynb',
 'DLEL_6_Softmax_Classifier.ipynb',
 'zoo.csv']

In [6]:
# load zoo dataset
xy = np.loadtxt(fname = './zoo.csv', delimiter = ',')
x_data = xy[:,0:-1]
y_data = xy[:,[-1]]
nb_classes = 7
print(x_data.shape, y_data.shape)

(101, 16) (101, 1)


In [7]:
X = tf.placeholder(dtype = tf.float32, shape = [None,x_data.shape[1]])
Y = tf.placeholder(dtype = tf.int32, shape = [None, 1])

Y_one_hot = tf.one_hot(Y, nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, shape = [-1,nb_classes])
print(Y_one_hot)

W = tf.Variable(initial_value = tf.random_normal(shape = [x_data.shape[1], nb_classes]), name = 'weight')
b = tf.Variable(initial_value = tf.random_normal(shape = [nb_classes]), name = 'bias')

Tensor("Reshape:0", shape=(?, 7), dtype=float32)


In [8]:
# cross entropy cost/loss
logits = tf.matmul(X, W) + b  #pre-activation
hypothesis = tf.nn.softmax(logits)
tmp_cost = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y_one_hot)
cost = tf.reduce_mean(tmp_cost)
opt = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize(cost)

# prediction & accuracy
prediction = tf.argmax(hypothesis, axis = 1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_one_hot, axis = 1), prediction), dtype = tf.float32))

In [9]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(2001):
        sess.run(opt, feed_dict = {X : x_data, Y : y_data})
        if step % 200 == 0:
            loss, acc = sess.run(fetches = [cost, accuracy], feed_dict = {X : x_data, Y : y_data})
            print('step : {:5}, loss : {:.4f}, acc : {:.2%}'.format(step, loss, acc))
            
    # Let's see if we can predict (임의의 10개를 뽑아서 예측 결과를 확인)
    pred = sess.run(prediction, feed_dict = {X : x_data})
    for p, y in np.asarray(list(zip(pred, y_data.flatten())))[list(np.random.randint(low = 0, high = y_data.shape[0], size = 10))]:
        print('[{}] Prediction : {} True Y : {}'.format(p == int(y), p, int(y)))

step :     0, loss : 7.9944, acc : 22.77%
step :   200, loss : 2.7889, acc : 27.72%
step :   400, loss : 1.6963, acc : 47.52%
step :   600, loss : 1.1950, acc : 57.43%
step :   800, loss : 0.9330, acc : 73.27%
step :  1000, loss : 0.7769, acc : 77.23%
step :  1200, loss : 0.6701, acc : 78.22%
step :  1400, loss : 0.5901, acc : 82.18%
step :  1600, loss : 0.5268, acc : 84.16%
step :  1800, loss : 0.4751, acc : 86.14%
step :  2000, loss : 0.4317, acc : 89.11%
[True] Prediction : 1.0 True Y : 1
[True] Prediction : 0.0 True Y : 0
[True] Prediction : 0.0 True Y : 0
[True] Prediction : 0.0 True Y : 0
[True] Prediction : 0.0 True Y : 0
[False] Prediction : 6.0 True Y : 5
[True] Prediction : 0.0 True Y : 0
[True] Prediction : 0.0 True Y : 0
[True] Prediction : 0.0 True Y : 0
[True] Prediction : 0.0 True Y : 0
